<a href="https://colab.research.google.com/github/mohyunyang/MyStudy/blob/master/ML_Ensemble%2CRandomForest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 정형데이터와 비정형 데이터
- 정형데이터는 주로 프로그래머가 다루는 데이터로서, 어떤 구조로 되어 있음(CSV, Excel에 저장하기 쉬움)
- 비정형데이터는 데이터베이스나 엑셀로 표현하기 어려움, 책의 글과 같은 텍스트데이터, 디지털카메라로 찍은 사진, 핸드폰으로 듣는 디지털음악 등등

### Ensemble learning (앙상블 학습)
- 정형데이터를 다루는데 가장 뛰어난 성과를 내는 알고리즘
- 이 알고리즘은 대부분 결정트리를 기반함
- 비정형데이터는 규칙성을 찾기 어려워 전통적인 머신러닝 방법으로는 모델을 만들기 까다로움, 해서 신경망 알고리즘으로 모델을 만들 수 있음)

# Random Forest (랜덤 포레스트)
- 앙상블 학습의 대표주자로 안정적인 성능 덕분에 널리 사용됨
- 결정트리를 랜덤하게 만들어 결정트리(나무)의 숲을 만듦
- 각 결정트리의 예측을 사용해 최종 예측을 만듦


> Bootstrap sample (부트스트랩 샘플) : 훈련데이터에서 랜덤하게 샘플을 추출하여 훈련데이터를 만드는데 이 때 한 샘플이 중복될 수도 있음 ex) 1개를 뽑고 다시 가방에 넣은다음 다시 뽑음



In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
wine = pd.read_csv('http://bit.ly/wine_csv_data')
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()
train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42
)

cross_validate() 함수를 사용해 교차검증 수행하기

In [8]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier     # RandomForestClassifier는 기본적으로 100개의 결정트리를 사용함
rf = RandomForestClassifier(n_jobs=-1, random_state=42) # n_jobs = -1 로 설정하여 최대한 병렬로 교차검증 수행 / 모든 CPU 코어를 사용하는게 좋음
scores = cross_validate(rf, train_input, train_target,  # return_train_score=True, 검증점수 뿐아니라 훈련세트에 대한 점수도 같이 반환함 (기본값은 False로 되있음)
                       return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

# 훈련세트에 다소 과대적합한 결과(여기서는 알고리즘을 조사하는것이 목적이므로 매개변수를 더 조정하지 않도록하자)

0.9973541965122431 0.8905151032797809


In [9]:
rf.fit(train_input, train_target)
print(rf.feature_importances_)

# 랜덤포레스트의 특성중요도는 각 결정트리의 특성중요도를 취합한 것임
# 다음은 랜덤포레스트 모델을 훈련세트에 훈련한 후 특성 중요도를 출력한 값이다.
# [알콜도수, 당도, pH]
# [0.12345626, 0.89892934,0.0079144] 다음은 결정트리의 특성중요도이다
# 분석해보자면 당도의 중요도 감소하고 나머지는 상승했다.
# 이런 이유는 랜덤 포레스트가 특성의 일부를 랜덤하게 선택하여 결정트리를 훈련하기 때문이다.
# 결과 하나의 특성에 과도하게 집중하지 않고 좀 더 많은 특성이 훈련에 기여할 기회를 얻는다
# 이는 과대적합을 줄이고 일반화 성능을 높이는데 도움된다.

[0.23167441 0.50039841 0.26792718]


#### OOB (out of bag)
부트스트랩 샘플에 포함되지 않고 남는 샘플
- 이 남는 샘플은 부트스트랩 샘플로 훈련한 결정트리를 평가할 수 있음
- 검증세트의 역할을 함





In [12]:
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)
rf.fit(train_input, train_target)
print(rf.oob_score_)

# 교차검증에서 얻은 점수와 매우 비슷한 결과를 얻음
# oob 점수를 사용하면 교차검증을 대신할 수 있어서 결과적으로 훈련세트에 더 많은 샘플을 사용할 수 있음

0.8934000384837406


# Extra Trees (엑스트라 트리)
- 기본적으로 100개의 결정트리를 훈련
- 랜덤포레스트와 동일하게 결정트리가 제공하는 대부분의 매개변수를 지원
- 전체 특성 중에 일부 특성을 랜덤하게 선택하여 노드를 분할하는데 사용
- 랜덤포레스트와 엑스트라 트리의 차이점은 부트스트랩 샘플을 사용하지 않는다
- 노드를 분할 할 때 가장 좋은 분할을 찾는 것이 아닌, 무작위로 분할 함

In [13]:
from sklearn.ensemble import ExtraTreesClassifier
et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
score = cross_validate(et, train_input, train_target,
                       return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(score['test_score']))

# 보통 엑스트라 트리가 무작위성이 좀 더 크기 때문에 랜덤포레스트보다 더 많은 결정트리를 훈련해야함
# 그러나 랜덤하게 노드를 분할 하기 때문에 빠른 계산속도가 장점이다

0.9973541965122431 0.8887848893166506


In [14]:
et.fit(train_input, train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


# gradient boosting (그레이디언트 부스팅)
깊이가 얕은 결정트리를 사용하여 이전 트리의 오차를 보완하는 방식으로 앙상블 하는 방법
- GrandientBoostingClassifier 는 기본적으로 깊이가 3인 결정트리를 100개 사용
- 깊이가 얕은 결정트리를 사용하기 때문에 과대적합에 강하고 일반적으로 높은 일반화성능을 기대할 수 있음
- 경사하강법을 사용하여 트리를 앙상블에 추가함 / 분류에서는 로지스틱 손실함수를, 회귀에서는 평균제곱오차함수를 사용
- 경실하강법이 손실함수를 산으로 정의하고 가장 낮은 곳을 찾아 내려오는 과정(모델의 가중치와 절편을 조금씩바꾸며)이라면 그레이디언트 부스팅은 결정트리를 계속 추가하면서 가장 낮은 곳을 찾아 이동함

In [20]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target,
                        return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))
# 과대적합이 되지 않음. 그레이디언트 부스팅은 결정트리의 개수를 늘려도 과대적합에 매우 강함
# 학습률을 증가시키고 트리의 개수를 늘리면 조금 더 성능이 향상될 수 있음

0.8881086892152563 0.8720430147331015


In [22]:
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2,  # 결정트리개수를 500개로 늘렸지만 과대적합을 잘 억제하는 중 # learning_rate=0.1 이 기본 값
                                random_state=42)
score = cross_validate(gb, train_input, train_target,
                       return_train_score=True, n_jobs= -1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


In [23]:
gb.fit(train_input, train_target)
print(gb.feature_importances_)

[0.15872278 0.68010884 0.16116839]


# Histogram-based Gradient Boosting (히스토그램 기반 그레이디언트 부스팅)
HistGradientBoostingClassifier
- 정형데이터를 다루는 머신러닝 알고리즘 중 가장 인기
- 히스토그램 기반 그레이디언트 부스팅은 먼저 입력특성을 256개 구간으로 나눔
- 따라서 노드를 분할 할 때 최적의 분할을 매우 빠르게 찾을 수 있음
- 히스토그램 기반 그레이디언트 부스팅은 256개의 구간 중에서 하나를 떼어 놓고 누락된 값을 위해서 사용
- 따라서 입력에 누락된 특성이 있더라도 이를 따로 전처리 할 필요가 없음
- 기본 매개변수에서 안정적인 성능을 얻을 수 있음

In [24]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target,
                        return_train_score=True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

/usr/local/lib/python3.7/dist-packages/sklearn/experimental/enable_hist_gradient_boosting.py:17: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  "Since version 1.0, "


0.9321723946453317 0.8801241948619236


In [25]:
# 과대적합을 잘 억제하면서 그레이디언트 부스팅보다 조금 더 높은 성능을 제공
# 특성의 중요도 확인
from sklearn.inspection import permutation_importance

hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target,
                                n_repeats=10, random_state=42, n_jobs= -1)  # n_repeats = 의 기본값은 5
print(result.importances_mean)

# importances, importances_mean, importances_std를 담고 있음

[0.08876275 0.23438522 0.08027708]


In [26]:
result = permutation_importance(hgb, test_input, test_target,
                                n_repeats=10, random_state=42, n_jobs=-1)
print(result.importances_mean)
# 당도에 더 집중하고 있군..
# 이런 모델을 실전에 투입했을 시 어떤 특성에 관심을 둘지 예상할 수 있음

[0.05969231 0.20238462 0.049     ]


In [27]:
hgb.score(test_input, test_target) # 테스트세트에서 성능을 최종적으로 확인하기

0.8723076923076923

In [28]:
from xgboost import XGBClassifier
xgb = XGBClassifier(tree_method='hist', random_state=42)
scores = cross_validate(xgb,train_input, train_target,
                        return_train_score=True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8824322471423747 0.8726214185237284


In [30]:
from lightgbm import LGBMClassifier
lgb = LGBMClassifier(random_state=42)
scores = cross_validate(lgb, train_input, train_target,
                        return_train_score=True, n_jobs= -1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9338079582727165 0.8789710890649293


#### 싸이킷런
1. 랜덤포레스트 : 부트스트랩 샘플사용, 대표 앙상블 학습 알고리즘

> 성능이 좋고 안정적임, 랜덤포레스트는 결정트리를 훈련하기 위해 부트스트랩 샘플을 만들고 전체 특성 중 일부를 랜덤하게 선택하여 결정트리를 만듦

2. 엑스트라 트리 : 결정트리의 노드를 랜덤하게 분할함

> 렌덤포레스트와 매우 비슷하지만 부트스트랩샘플을 사용하지 않고 노드를 분할할 때 최선이 아니라 랜덤하게 분할함, 랜덤포레스트보다 훈련속도가 빠르지만 보통 더 많은 트리필요함 


3. 그레이디언트 부스팅 : 이전 트리의 손실을 보완하는 식으로 얕은 결정트리를 연속하여 추가함


> 깊이가 얕은 트리를 연속적으로 추가하여 손실함수를 최소화하는 앙상블방법이다. 성능은 뛰어나지만 병렬로 훈련할 수 없기에 랜덤포레스트나 엑스트라트리보다 훈련속도가 조금 느림, 학습률 매개변수를 조정하여 모델의 복잡도를 제어가능, 학습률 매개변수가 크면 복잡하고 훈련세트에 과대적합된 모델을 얻을 수 있다


4. 히스토그램 기반 그레이디언트 부스팅 : 훈련데이터를 256개 정수구간으로 나누어 빠르고 높은 성능을 냄

